In [ ]:
import tensorflow as tf
from tensorflow.contrib import layers
from tensorflow.contrib import rnn  # rnn stuff temporarily in contrib, moving back to code in TF 1.1
import os
import time
import math
import numpy as np
import my_txtutils as txt
tf.set_random_seed(0)

# model parameters
#
# Usage:
#   Training only:
#         Leave all the parameters as they are
#         Disable validation to run a bit faster (set validation=False below)
#         You can follow progress in Tensorboard: tensorboard --log-dir=log
#   Training and experimentation (default):
#         Keep validation enabled
#         You can now play with the parameters anf follow the effects in Tensorboard
#         A good choice of parameters ensures that the testing and validation curves stay close
#         To see the curves drift apart ("overfitting") try to use an insufficient amount of
#         training data (shakedir = "shakespeare/t*.txt" for example)
#
SEQLEN = 30
BATCHSIZE = 200
ALPHASIZE = txt.ALPHASIZE
INTERNALSIZE = 512
NLAYERS = 3
learning_rate = 0.001  # fixed learning rate
dropout_pkeep = 0.8    # some dropout

# load data, either shakespeare, or the Python source of Tensorflow itself
shakedir = "/Users/muratyalcin/Desktop/Data Science/Upwork/2/tensorflow-rnn-shakespeare-master/shakespeare_input.txt"
#shakedir = "../tensorflow/**/*.py"
codetext, valitext, bookranges = txt.read_data_files(shakedir, validation=True)

# display some stats on the data
epoch_size = len(codetext) // (BATCHSIZE * SEQLEN)
txt.print_data_stats(len(codetext), len(valitext), epoch_size)

#
# the model (see FAQ in README.md)
#
lr = tf.placeholder(tf.float32, name='lr')  # learning rate
pkeep = tf.placeholder(tf.float32, name='pkeep')  # dropout parameter
batchsize = tf.placeholder(tf.int32, name='batchsize')

# inputs
X = tf.placeholder(tf.uint8, [None, None], name='X')    # [ BATCHSIZE, SEQLEN ]
Xo = tf.one_hot(X, ALPHASIZE, 1.0, 0.0)                 # [ BATCHSIZE, SEQLEN, ALPHASIZE ]
# expected outputs = same sequence shifted by 1 since we are trying to predict the next character
Y_ = tf.placeholder(tf.uint8, [None, None], name='Y_')  # [ BATCHSIZE, SEQLEN ]
Yo_ = tf.one_hot(Y_, ALPHASIZE, 1.0, 0.0)               # [ BATCHSIZE, SEQLEN, ALPHASIZE ]
# input state
Hin = tf.placeholder(tf.float32, [None, INTERNALSIZE*NLAYERS], name='Hin')  # [ BATCHSIZE, INTERNALSIZE * NLAYERS]

# using a NLAYERS=3 layers of GRU cells, unrolled SEQLEN=30 times
# dynamic_rnn infers SEQLEN from the size of the inputs Xo

# How to properly apply dropout in RNNs: see README.md
cells = [rnn.GRUCell(INTERNALSIZE) for _ in range(NLAYERS)]
# "naive dropout" implementation
dropcells = [rnn.DropoutWrapper(cell,input_keep_prob=pkeep) for cell in cells]
multicell = rnn.MultiRNNCell(dropcells, state_is_tuple=False)
multicell = rnn.DropoutWrapper(multicell, output_keep_prob=pkeep)  # dropout for the softmax layer

Yr, H = tf.nn.dynamic_rnn(multicell, Xo, dtype=tf.float32, initial_state=Hin)
# Yr: [ BATCHSIZE, SEQLEN, INTERNALSIZE ]
# H:  [ BATCHSIZE, INTERNALSIZE*NLAYERS ] # this is the last state in the sequence

H = tf.identity(H, name='H')  # just to give it a name

# Softmax layer implementation:
# Flatten the first two dimension of the output [ BATCHSIZE, SEQLEN, ALPHASIZE ] => [ BATCHSIZE x SEQLEN, ALPHASIZE ]
# then apply softmax readout layer. This way, the weights and biases are shared across unrolled time steps.
# From the readout point of view, a value coming from a sequence time step or a minibatch item is the same thing.

Yflat = tf.reshape(Yr, [-1, INTERNALSIZE])    # [ BATCHSIZE x SEQLEN, INTERNALSIZE ]
Ylogits = layers.linear(Yflat, ALPHASIZE)     # [ BATCHSIZE x SEQLEN, ALPHASIZE ]
Yflat_ = tf.reshape(Yo_, [-1, ALPHASIZE])     # [ BATCHSIZE x SEQLEN, ALPHASIZE ]
loss = tf.nn.softmax_cross_entropy_with_logits(logits=Ylogits, labels=Yflat_)  # [ BATCHSIZE x SEQLEN ]
loss = tf.reshape(loss, [batchsize, -1])      # [ BATCHSIZE, SEQLEN ]
Yo = tf.nn.softmax(Ylogits, name='Yo')        # [ BATCHSIZE x SEQLEN, ALPHASIZE ]
Y = tf.argmax(Yo, 1)                          # [ BATCHSIZE x SEQLEN ]
Y = tf.reshape(Y, [batchsize, -1], name="Y")  # [ BATCHSIZE, SEQLEN ]
train_step = tf.train.AdamOptimizer(lr).minimize(loss)

# stats for display
seqloss = tf.reduce_mean(loss, 1)
batchloss = tf.reduce_mean(seqloss)
accuracy = tf.reduce_mean(tf.cast(tf.equal(Y_, tf.cast(Y, tf.uint8)), tf.float32))
loss_summary = tf.summary.scalar("batch_loss", batchloss)
acc_summary = tf.summary.scalar("batch_accuracy", accuracy)
summaries = tf.summary.merge([loss_summary, acc_summary])

# Init Tensorboard stuff. This will save Tensorboard information into a different
# folder at each run named 'log/<timestamp>/'. Two sets of data are saved so that
# you can compare training and validation curves visually in Tensorboard.
timestamp = str(math.trunc(time.time()))
summary_writer = tf.summary.FileWriter("log/" + timestamp + "-training")
validation_writer = tf.summary.FileWriter("log/" + timestamp + "-validation")

# Init for saving models. They will be saved into a directory named 'checkpoints'.
# Only the last checkpoint is kept.
if not os.path.exists("checkpoints"):
    os.mkdir("checkpoints")
saver = tf.train.Saver(max_to_keep=1000)

# for display: init the progress bar
DISPLAY_FREQ = 50
_50_BATCHES = DISPLAY_FREQ * BATCHSIZE * SEQLEN
progress = txt.Progress(DISPLAY_FREQ, size=111+2, msg="Training on next "+str(DISPLAY_FREQ)+" batches")

# init
istate = np.zeros([BATCHSIZE, INTERNALSIZE*NLAYERS])  # initial zero input state
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
step = 0

# training loop
for x, y_, epoch in txt.rnn_minibatch_sequencer(codetext, BATCHSIZE, SEQLEN, nb_epochs=10):

    # train on one minibatch
    feed_dict = {X: x, Y_: y_, Hin: istate, lr: learning_rate, pkeep: dropout_pkeep, batchsize: BATCHSIZE}
    _, y, ostate = sess.run([train_step, Y, H], feed_dict=feed_dict)

    # log training data for Tensorboard display a mini-batch of sequences (every 50 batches)
    if step % _50_BATCHES == 0:
        feed_dict = {X: x, Y_: y_, Hin: istate, pkeep: 1.0, batchsize: BATCHSIZE}  # no dropout for validation
        y, l, bl, acc, smm = sess.run([Y, seqloss, batchloss, accuracy, summaries], feed_dict=feed_dict)
        txt.print_learning_learned_comparison(x, y, l, bookranges, bl, acc, epoch_size, step, epoch)
        summary_writer.add_summary(smm, step)

    # run a validation step every 50 batches
    # The validation text should be a single sequence but that's too slow (1s per 1024 chars!),
    # so we cut it up and batch the pieces (slightly inaccurate)
    # tested: validating with 5K sequences instead of 1K is only slightly more accurate, but a lot slower.
    if step % _50_BATCHES == 0 and len(valitext) > 0:
        VALI_SEQLEN = 1*1024  # Sequence length for validation. State will be wrong at the start of each sequence.
        bsize = len(valitext) // VALI_SEQLEN
        txt.print_validation_header(len(codetext), bookranges)
        vali_x, vali_y, _ = next(txt.rnn_minibatch_sequencer(valitext, bsize, VALI_SEQLEN, 1))  # all data in 1 batch
        vali_nullstate = np.zeros([bsize, INTERNALSIZE*NLAYERS])
        feed_dict = {X: vali_x, Y_: vali_y, Hin: vali_nullstate, pkeep: 1.0,  # no dropout for validation
                     batchsize: bsize}
        ls, acc, smm = sess.run([batchloss, accuracy, summaries], feed_dict=feed_dict)
        txt.print_validation_stats(ls, acc)
        # save validation data for Tensorboard
        validation_writer.add_summary(smm, step)

    # display a short text generated with the current weights and biases (every 150 batches)
    if step // 3 % _50_BATCHES == 0:
        txt.print_text_generation_header()
        ry = np.array([[txt.convert_from_alphabet(ord("K"))]])
        rh = np.zeros([1, INTERNALSIZE * NLAYERS])
        for k in range(1000):
            ryo, rh = sess.run([Yo, H], feed_dict={X: ry, pkeep: 1.0, Hin: rh, batchsize: 1})
            rc = txt.sample_from_probabilities(ryo, topn=10 if epoch <= 1 else 2)
            print(chr(txt.convert_to_alphabet(rc)), end="")
            ry = np.array([[rc]])
        txt.print_text_generation_footer()

    # save a checkpoint (every 500 batches)
    if step // 10 % _50_BATCHES == 0:
        saved_file = saver.save(sess, 'checkpoints/rnn_train_' + timestamp, global_step=step)
        print("Saved file: " + saved_file)

    # display progress bar
    progress.step(reset=step % _50_BATCHES == 0)

    # loop state around
    istate = ostate
    step += BATCHSIZE * SEQLEN

Loading file /Users/muratyalcin/Desktop/Data Science/Upwork/2/tensorflow-rnn-shakespeare-master/shakespeare_input.txt
Training text size is 4.36MB with 0.00KB set aside for validation. There will be 762 batches per epoch
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.


   0 (epoch 0) shakespeare_input.txt │ First Citizen:\Before we proce │                                │ loss: 4.50328
  30 (epoch 0) shakespeare_input.txt │ :\O,'tis Marcius!\Let's fetch  │                                │ loss: 4.52118
  60 (epoch 0) shakespeare_input.txt │ f the nobles.\\SICINIUS:\I wis │                                │ loss: 4.54797
  90 (epoch 0) shakespeare_input.txt │ u not set them on?\\MENENIUS:\ │                                │ loss: 4.52342
 120 (epoch 0) shakespeare_input.txt │ temples with the shows of peac │                                │ loss: 4.

 nsien ono ttteidi
oerrdr


d 
ieaho a
 anoritn
 
roe 
toae o

ri oenon
t r   eerdtitnaor
ao aa te
o ta oa
e
oene ed
 
andnrrtri
r

dr
edte
hoe  dorhaoediediaeootii
 da diootidaa reado
etnaarrr

ie
 rhoih
thratr

r i trnnrirneennohror r on
oiaatran
nroa
tnaod oitee
  o aton
edaerorrdie  aaiiertrterrnai
 ehan n
tr neirieh 
dn  roe drtnd t
ote
trne i to
rtonre
 oioatoar r eronotnnri
teoeiteat ddn rnia
nde t iadodr
et etnrnerereornie re  rtta

oiatdodertrto
eeaeo
oeaore
ta er rr
ti
eiar
 e
 rrhneea  nndhaeaetnorentaa
 ite 
itin
d i otnni derorrtnniadnr t dr aa

rodooe
dooer ro
atrn nrrieron
 tdir aiaenierter idoii it aadiei
eaot i
 iot    rididint e r  rntenidooartoao
 reo aoo  id nntetor rata
aoinnai
aa
th taotin
ateat

 ohoaotdddaotr i
retia
eo 
o
d
aoitnr  anna

irro t
oi t
reneroeoteartir
n
r
id
r e o
ondehenraortoe dt  dreoai reidd
ete irarr ttdedo od 
toaoianrtai
onntttiter ior
dnno  rdtrroieeitao
e
aaa idet
to redodn
taarrd eao
nn
ndoo
n d
 e
tr
 aidteorrinn arndtn
└───────────────

600000 (epoch 0) shakespeare_input.txt │ ever\Appear in your impediment │ ee    oo   te te   te e        │ loss: 3.15600
600030 (epoch 0) shakespeare_input.txt │ Condemning some to death, and  │           he   te ee     ho  t │ loss: 2.72183
600060 (epoch 0) shakespeare_input.txt │ of\ingrateful injury; to repor │ o  h         he      hoeee ee  │ loss: 3.38083
600090 (epoch 0) shakespeare_input.txt │ rance; if none, awake\Your dan │       ho te    hoe   \ h  he   │ loss: 3.10196
600120 (epoch 0) shakespeare_input.txt │  in the name o' the people\And │ ho eee te   te tee toe ee \\   │ loss: 2.73250
600150 (epoch 0) shakespeare_input.txt │ ut insolent,\O'ercome with pri │ e ee              e  toe  tee  │ loss: 3.06417
600180 (epoch 0) shakespeare_input.txt │ at brought you forth this boy, │   hoe     he  hee   ee   toe   │ loss: 2.75077
600210 (epoch 0) shakespeare_input.txt │ bear the corse, and set it dow │     oo  toe    ho  te  he ee   │ loss: 2.54524
600240 (epoch 0) shakespeare_inp

604920 (epoch 0) shakespeare_input.txt │ horse for my mistress, or any\ │ o    oe  te tee       ho te  \ │ loss: 2.72799
604950 (epoch 0) shakespeare_input.txt │  high, and we outwear the day. │ ao    ho  to te      hee te  \ │ loss: 2.67692
604980 (epoch 0) shakespeare_input.txt │ s\suffered under that shape, I │   hh      ho    hee  hee e  he │ loss: 2.95631
605010 (epoch 0) shakespeare_input.txt │ ish.\\BURGUNDY:\Is she not apt │     \\o AEE:::::  oe  ee  ooe  │ loss: 3.00014
605040 (epoch 0) shakespeare_input.txt │ uth is, Sir John, you live in  │    ee  hoe te    ho  te   te t │ loss: 3.05173
605070 (epoch 0) shakespeare_input.txt │  and I could discern no part o │ ho  teooe   te      te tee  te │ loss: 2.86917
605100 (epoch 0) shakespeare_input.txt │ ot read to burst,--\well, swee │ e ee   te eee       h    hoe   │ loss: 3.00233
605130 (epoch 0) shakespeare_input.txt │ eigh'd\What wrongs our arms ma │       \ho  hoe    ho  te   hee │ loss: 2.95863
605160 (epoch 0) shakespeare_inp

904800 (epoch 0) shakespeare_input.txt │  the liberty of gazing.\Which  │ the tese e  tr toren  \\hete t │ loss: 2.39837
904830 (epoch 0) shakespeare_input.txt │  of it.\\ELY:\I'll wait upon y │ tr tn \\ARIRNI t  torl trerdto │ loss: 2.65135
904860 (epoch 0) shakespeare_input.txt │ e of Exeter,\Enlarge the man c │  tn toarh e Todene rthe terdto │ loss: 2.57806
904890 (epoch 0) shakespeare_input.txt │  work ish ill done;\it ish giv │ tore tn etn  tor  \Tn tn etote │ loss: 2.55154
904920 (epoch 0) shakespeare_input.txt │ \I know him to be valiant.\\Co │ Woto r tes thrte terenn  \\AAr │ loss: 2.57089
904950 (epoch 0) shakespeare_input.txt │ As one man more, methinks, wou │ n trd terdtere \to  en    tord │ loss: 2.16979
904980 (epoch 0) shakespeare_input.txt │ ,\Of knights, esquires, and ga │ \To te t e   tr trte   tnd tor │ loss: 2.73907
905010 (epoch 0) shakespeare_input.txt │ a fair French city\for one fai │ ntorn toe  eetenh Toretr  torn │ loss: 2.82406
905040 (epoch 0) shakespeare_inp

...

I do long angel to there and out awoll their sword,
So have to dispair out and a fotlenged, they take my trath
Of thy house, wide alrocker trombs and merry.
I'll tell a wathress of the die al the sport shall pray your sons:
That shulps a morty coors to that if I would
have heands honest thy free affaint,
His partasor of her stare and fear our;
Forsunder what at its with sour highnass off on his-servant,
Where is we dost this groath, as I am dishion' tean and more,
To he are too befale his porconious dume,
And morrings falls fair pisted cliff-be fair as hyre, for you
treat homes of the shake of this greater dead,
Hy thing I think you are all to do, for that the rest of all;
I cunlothing hap out of the percy to her strayge,
That is no make of them of lood,
And it we shall sun such a pate mine my highness in the compassion of my fangers, for
yevenged of his histers water shoind on the
would say is my strarge feind, who that yot be
song me so sing; how now since I have pass,
Wrich hou 
└──

8700000 (epoch 1) shakespeare_input.txt │  on: yet, by the faith of men, │ tf  tot  mu the sotrh,of tyn\\ │ loss: 1.44591
8700030 (epoch 1) shakespeare_input.txt │ e Capitol.\\All:\We will so: a │  sosioin,\\CNl:\Hhlsill ne  tn │ loss: 1.71922
8700060 (epoch 1) shakespeare_input.txt │ ok, I am going:\Commend me to  │ k  t wm noong \Iomeond te to t │ loss: 1.24783
8700090 (epoch 1) shakespeare_input.txt │ looking as it were--would I we │ eok ng tn tt iele -\hrld h hil │ loss: 1.58299
8700120 (epoch 1) shakespeare_input.txt │  Rome,\Loved me above the meas │ tome,\Tere  te t oue the faano │ loss: 1.54251
8700150 (epoch 1) shakespeare_input.txt │ Got my lord chamberlain his li │ o  se lord,ooarber ynn tim sof │ loss: 1.69479
8700180 (epoch 1) shakespeare_input.txt │ \Which here you urge to prove  │ Thech sa e tou sneedth trove t │ loss: 1.36073
8700210 (epoch 1) shakespeare_input.txt │ I done a good day's work:\You  │  ho e tncood say s ford.\Iou s │ loss: 1.60685
8700240 (epoch 1) shakes

9000000 (epoch 1) shakespeare_input.txt │ ort his honours\From where he  │ rt tim senour..Toom thane te s │ loss: 1.21304
9000030 (epoch 1) shakespeare_input.txt │ \For they do prank them in aut │ Aor Ihe  ao nroyc thee tn t lh │ loss: 1.40143
9000060 (epoch 1) shakespeare_input.txt │ xecution\Of what we chance to  │ ecution Tf that te waange th t │ loss: 1.26591
9000090 (epoch 1) shakespeare_input.txt │ \But, more of thy news?\\Third │ Tut  syre tf the caws \\BIird  │ loss: 1.24791
9000120 (epoch 1) shakespeare_input.txt │  but do not say\For that 'Forg │ aut to yot bey Tor thet ttoree │ loss: 1.55502
9000150 (epoch 1) shakespeare_input.txt │ s widow sister,\I will perform │ htitew thnter,\Tnwill nrrcorm  │ loss: 1.51231
9000180 (epoch 1) shakespeare_input.txt │ eep'd in the faultless blood o │ np d tn the cocltsyss orood.tf │ loss: 1.38113
9000210 (epoch 1) shakespeare_input.txt │ teous love\Doth cherish you an │ hrns sive.To h toaeit  tourtrd │ loss: 1.63501
9000240 (epoch 1) shakes

My lord: the compon tulls I know
Why, what doth said you the commons a kind?

GLOUCESTER:
My liegese heep heart.
So do wht stould attend upon thee.

SOTFLRCKE:
Ay, methand, thou wast been andemy
like a man dod bl strong as more wrong,
Turn'd friends he did, are mine end;
Or it to diar.

LOND POLONIOS:
Well, I will make thee will, I cannot be to pet.

PETES:
Here, madam, a delivion he is times?
Whom he alent his fine, you dram thee a bream so,
The pencle false of the balls.

HORATIO:
Why, that's this infortunes? What shalt, you love thee
My lord? what warron to the say,
And let them then to bein half-stold in
Frinch and cour emption woold as abladd;
But tell me. There's set a parerance,
Troublother, sword your lord,
I'll seek to stupfice of a waming:
Our wite heard to achise thy man
And said to part werch of his sons,
Becondect yaugh and didgeses, with his
brother, carrity slave; hold him like
A common than a bony-seaven and
brinking a great forsetes whomes
the parson.

SALUNNINE:
I 
└─

9600000 (epoch 2) shakespeare_input.txt │ ixt doing them neither\good no │ lt to ng toe  towgher Aood mot │ loss: 1.49923
9600030 (epoch 2) shakespeare_input.txt │ We let the people know't.\\MEN │ h soa the braple onow s \\FANE │ loss: 1.32153
9600060 (epoch 2) shakespeare_input.txt │ o you prate of service?\\BRUTU │  you sroy  yu moevece \\CRUTUS │ loss: 1.18990
9600090 (epoch 2) shakespeare_input.txt │ r, if\He could have temporized │   tf Ie samld nave shlpereted  │ loss: 1.67680
9600120 (epoch 2) shakespeare_input.txt │ se\The country, our dear nurse │ t The sourtry  afr ceat oomse  │ loss: 1.35414
9600150 (epoch 2) shakespeare_input.txt │  blood that let this blood fro │ ceood ooat Iot thes meood,ooom │ loss: 1.21628
9600180 (epoch 2) shakespeare_input.txt │  yourself.\\QUEEN MARGARET:\Po │ tour elf,\\CUEEN MARGARET:\Iro │ loss: 0.70954
9600210 (epoch 2) shakespeare_input.txt │ entleman\Lately attendant on t │ ntleman,Wode y tn end nt tf th │ loss: 1.32746
9600240 (epoch 2) shakes